# Preamble

In [ ]:
# json
import json

# Pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio

import matplotlib.pyplot as plt
import librosa

import librosa.display
import IPython.display as ipd

import numpy as np
import pickle
import random

In [ ]:
import torch
import librosa
from torch_specinv import griffin_lim
from torch_specinv.metrics import spectral_convergence as SC

from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload

In [ ]:
from tqdm import tqdm
from sklearn import metrics

In [ ]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda=True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Load data

In [ ]:
from datasets.datasethandler import DatasetHandler 
datasetHandler = DatasetHandler()

# Validating

In [ ]:
def reportScore(y_true, y_pred):
    print("\tAccuracy:\t" + str(metrics.accuracy_score(y_true,y_pred)))
    print("\tPrecision:\t" + str(metrics.precision_score(y_true,y_pred)))
    print("\tRecall:   \t" + str(metrics.recall_score(y_true,y_pred)))
    print("\tF1-score:\t" + str(metrics.f1_score(y_true,y_pred)))
    #tn, fp, fn, tp = metrics.confusion_matrix(y_true,y_pred).ravel()

In [ ]:
def validate(model):
    valid_loader = model.get_dataloader("validation", shuffle=False) 
    model.eval()

    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in tqdm(valid_loader):
            x, labels = data['audio'].to(device), data['label'].to(device)
            outputs = model(x)

            y_pred.append(torch.max(outputs.data, 1)[1].item())
            y_true.append(labels.item())
            
    reportScore(y_true, y_pred)

# Training

In [ ]:
def test():
    hparams = {
        "batch_size": 24,
        "learning_rate": 0.002,
        "weight_decay": 0.01,
        "lr_decay": 0.95
    }

    model = AudioCRNNPLModule(hparams, config=cfg).to(device)
    model.float()

    datasetHandler.load(model, "training")
    datasetHandler.load(model, "validation")

    x = next(iter(model.get_dataloader("training", batch_size=32)))
    x = x['audio'].to(device)
    
    return model(x)

In [ ]:
torch.cuda.empty_cache()

hparams = {
    "batch_size": 24,
    "learning_rate": 0.002,
    "weight_decay": 0.01,
    "lr_decay": 0.95
}

model = AudioCRNNPLModule(hparams, config=cfg).to(device)
model.float()

datasetHandler.load(model, "training")
datasetHandler.load(model, "validation")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

for epoch in range(12):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(model.get_dataloader("training", batch_size=32)):
        x, labels = data['audio'].to(device), data['label'].to(device)
        
        model.train()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(x)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # track statistics
        running_loss += loss.item()
    #print(outputs[:10])
    
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / (i+1)))
    validate(model)
    
print('Finished Training')

print("Validate:")
validate(model)

In [ ]:
validate(model)

In [ ]:
#model_state_dict_path = "/nfs/students/summer-term-2020/project-4/data/models/SpectrogramBasedCNN.pt"
#torch.save(model.state_dict(), model_state_dict_path)

# Backprop

In [ ]:
x = next(iter(model.get_dataloader("training", batch_size=32)))
x, labels = x['audio'].to(device), x['label'].to(device)

In [ ]:
model.train()
model.zero_grad()
x.requires_grad_()

outputs = model(x)

loss = nn.CrossEntropyLoss()(outputs, labels)
loss.backward()
print(x.grad)

In [ ]:
x + 0.001 * x.grad

In [ ]:
grad = x.grad
grad[grad!=grad] = 0

In [1]:
import torch

# Debugging dataset handler

In [3]:
x = torch.load("/nfs/students/summer-term-2020/project-4/data/models/crnn.pth")

In [ ]:
datasetHandler = DatasetHandler()

In [ ]:
from classification.models.M5 import M5PLModule

hparams = {
    "batch_size": 2,
    "learning_rate": 0.001,
    "weight_decay": 0,
    "lr_decay": 1
}

model = M5PLModule(hparams)

datasetHandler.load_datasets(model)

In [ ]:
for x in model.get_dataloader("training", batch_size=32):
    pass

In [ ]:
for x in model.get_dataloader("validation", batch_size=32):
    pass

In [ ]:
for x in model.get_dataloader("testing", batch_size=32):
    pass